In [62]:
import pandas as pd
import requests
import json

pd.options.display.max_columns = 30


In [63]:
def page_url(page=1):
    '''
    Uses default query specifications
    '''
    api_key  = "api_key=d2ba87d601823962c8cc04aa877278a1"
    discover_api = "https://api.themoviedb.org/3/discover/movie?"
    query = "&language=en-US&sort_by=popularity.desc&include_adult=false&page={}&vote_count.gte=500".format(page)
    url = discover_api + api_key + query
    return url

In [64]:
url = discover_api + api_key + default_query

def make_big_dataframe(max_pages):
    '''
    The API proides data in pages of 10 movies per url request. 
    This function iterates over each page adding the data to one big pandas df
    '''
    if (max_pages >= 214):
        print("max is 213")
        return
    
    frames = []
    
    for page_num in range(1,max_pages+1):
        url = page_url(page_num)
        r = requests.get(url)
        if r.status_code != 200:
            '''Something went wrong'''
            print("Error")
            return
        data = r.json()
        page_df = pd.json_normalize(data = data, record_path = "results") # normalize, as you convert to df
        frames.append(page_df)
       
    # concat dfs into one
    df = pd.concat(frames) # method takes array of df
    
    # process
    df = df.reset_index() # (else they restart at 19)
    df =df[["title",
            "genre_ids",
            "release_date",
            "overview",
            "popularity",
            "vote_count",
            "poster_path",
            "vote_average"]]
    
    return df  

test_frame = make_big_dataframe(10)
# functional    


In [65]:
test_frame

,title,genre_ids,release_date,overview,popularity,vote_count,poster_path,vote_average
0,Ad Astra,"[18, 878]",2019-09-17,"The near future, a time when both hope and har...",165.765,3802,/xBHvZcjRiWyobQ9kxBhO6B2dtRI.jpg,6.1
1,Sonic the Hedgehog,"[28, 35, 878, 10751]",2020-02-12,Based on the global blockbuster videogame fran...,117.793,4784,/aQvJ5WPzZgYVDrxLX4R6cLJCEaQ.jpg,7.5
2,Parasite,"[35, 18, 53]",2019-05-30,"All unemployed, Ki-taek's family takes peculia...",96.606,8123,/7IiTTgloJzvGI1TAYymCfbfl3vT.jpg,8.5
3,Avengers: Infinity War,"[28, 12, 878]",2018-04-25,As the Avengers and their allies have continue...,82.688,18663,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,8.3
4,The Invisible Man,"[27, 878, 53]",2020-02-26,When Cecilia's abusive ex takes his own life a...,94.348,2225,/5EufsDwXdY2CVttYOk2WtYhgKpa.jpg,7.2
...,...,...,...,...,...,...,...,...
195,Thor,"[28, 12, 14]",2011-04-21,"Against his father Odin's will, The Mighty Tho...",27.471,15015,/prSfAi1xGrhLQNxVSUFh61xQ4Qy.jpg,6.7
196,Alien: Covenant,"[27, 878]",2017-05-09,Bound for a remote planet on the far side of t...,25.225,5795,/zecMELPbU5YMQpC81Z8ImaaXuf9.jpg,5.9
197,The Mask,"[35, 80, 14, 10749]",1994-07-29,When timid bank clerk Stanley Ipkiss discovers...,34.309,6684,/yd3wjLLeQHJ3oRDy4wwiNNFVyLW.jpg,6.9
198,Mission: Impossible - Fallout,"[28, 12]",2018-07-13,"When an IMF mission ends badly, the world is f...",32.475,5100,/AkJQpZp9WoNdj7pLYSj1L0RcMMN.jpg,7.3


### Now the ML processing starts

In [44]:
 ## add a column with the concatenation of TITLE and OVERVIEW